<a href="https://colab.research.google.com/github/vermelho25/analitica/blob/main/Reto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB

# Cargar el archivo de excel
path = '/content/sample_data/1. BD2_CarCrash_Int.xlsx'
dataframe = pd.read_excel(path)

# Tomar San Francisco
dataframe2 = dataframe[dataframe['County'] == 'SAN FRANCISCO']

# Variables de interés
dataframe3 = dataframe2[['Weekday', 'ViolCat', 'Daylight', 'Month', 'CrashType', 'ClearWeather', 'Highway']]

# Convertir CrashType a valores numéricos según orden alfabético
dataframe3.loc[:, 'CrashType'] = dataframe3['CrashType'].astype('category').cat.codes

# Filtrar registros vacios
dataframe3.dropna(inplace=True)

# Definir variables de entrada y salida
XD = dataframe3[['Weekday', 'ViolCat', 'Daylight', 'Month', 'CrashType', 'ClearWeather']]
yd = dataframe3[['Highway']]

# Entrenar modelo Naive Bayes
mnb = GaussianNB()
mnb.fit(XD, yd.values.ravel())


###ELIMINAR
# Obtener información del modelo
ct = mnb.class_count_
print("Distribución de los datos:", ct)
print("Total de datos:", len(XD))
print("Porcentaje por categoría:", ct / len(XD))

# Calcular medias y desviaciones estándar por categoría
u = mnb.theta_
std = np.sqrt(mnb.var_)
print("Media por categoría:\n", u)
print("Desviación estándar por categoría:\n", std)


Distribución de los datos: [1580.  501.]
Total de datos: 2081
Porcentaje por categoría: [0.75925036 0.24074964]
Media por categoría:
 [[3.88417722 7.45696203 0.66265823 4.35189873 3.21772152 0.86265823]
 [3.9261477  4.1497006  0.62075848 6.69461078 2.27744511 0.79840319]]
Desviación estándar por categoría:
 [[1.95062542 3.52136997 0.47280262 2.3433645  1.87371962 0.34420782]
 [2.01603723 2.16198902 0.48519831 3.44163641 1.05765448 0.40119266]]


<ipython-input-1-ef370afec497>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe3.dropna(inplace=True)


In [ ]:
#1
num_datos = dataframe.shape[0]
num_datos_sf = dataframe3.shape[0]
print("El total de todos los datos, sin filtrar San Francisco es:", num_datos)
print("El total de todos los datos, filtrando San Francisco es:", num_datos_sf)

#2
print("Porcentaje por categoría:", ct / len(XD))

El total de todos los datos, sin filtrar San Francisco es: 112660
El total de todos los datos, filtrando San Francisco es: 2081
Porcentaje por categoría: [0.75925036 0.24074964]


In [ ]:
#3 Dias de la semana
print("Dias de la semana")
intervalos = dataframe3.groupby('Highway')['Weekday'].agg(['mean', 'std'])

limite_inferior = intervalos['mean'] - intervalos['std']
limite_superior = intervalos['mean'] + intervalos['std']

print(limite_inferior)
print(limite_superior)

#4 Violcat
print("Violcat")
intervalos2 = dataframe3.groupby('Highway')['ViolCat'].agg(['mean', 'std'])

limite_inferior2 = intervalos2['mean'] - intervalos2['std']
limite_superior2 = intervalos2['mean'] + intervalos2['std']

print(limite_inferior2)
print(limite_superior2)

#5 Crashtype
print("Crashtype")
intervalos3 = dataframe3.groupby('Highway')['CrashType'].agg(['mean', 'std'])

limite_inferior3 = intervalos3['mean'] - intervalos3['std']
limite_superior3 = intervalos3['mean'] + intervalos3['std']

print(limite_inferior3)
print(limite_superior3)




Dias de la semana
Highway
0    1.932934
1    1.908095
dtype: float64
Highway
0    5.83542
1    5.94420
dtype: float64
Violcat
Highway
0    3.934477
1    1.985551
dtype: float64
Highway
0    10.979447
1     6.313851
dtype: float64
Crashtype
Highway
0    1.343409
1    1.218734
dtype: object
Highway
0    5.092034
1    3.336157
dtype: object


Entre los hallazgos menciono que la media de weedays es aproximadamente 3,9. Lo cual indica que en promedio la mayoría de choques se da entre martes y miercoles (considerando un calendario donde el primer día es domingo) los límites superior e inferior siento que no me indican nada ya que los números en este caso solo son una decodificación, más los días de la semana no conllevan una escala. Crashtype que es otra variable codificada tuvo como promedio 3,2 para accidentes fuera de carretera y 2,27 para accidentes en carretera, lo que indica que los accidentes fueron de tipo entre B y C en promedio.
Y en cuanto al ViolCat, para los accidentes en carretera osciló en un valor de dispersión entre 1,98 y 6,3. Mientras que en los que fueron fuera de carretera osciló entre 3,93 y 10,97.



In [ ]:
from sklearn.metrics import confusion_matrix

ydp = mnb.predict(XD)
cm=confusion_matrix(yd,ydp)
VN=cm [0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

#6
Ex=(VN+VP)/(VN+FP+FN+VP)
print("la exactitud es:\n", Ex)

#7
Esp=VN/(VN+FP)
print("la especificidad es:\n", Esp)

la exactitud es:
 0.7890437289764536
la especificidad es:
 0.8639240506329114


La exactitud en términos generales es buena, del 78,9%. Podría ser mejor. En cuanto a especificidad, es bastante alta, significa que los verdaderos son bien identificados y el modelo sabe descartarlos correctamente en su gran mayoría.

In [ ]:
#8
print("Verdaderos negativos y verdaderos positivos:\n", cm)

Verdaderos negativos y verdaderos positivos:
 [[1365  215]
 [ 224  277]]


Tenemos 1365 Verdaderos positivos y 277 Verdaderos negativos

In [ ]:
#9
TE=(FN+FP)/(VN+FP+FN+VP)
print("la tasa de error es:\n", TE)

#10
Sen=VP/(VP+FN)
print("la sensibilidad es:\n", Sen)

#11
Pre=VP/(VP+FP)
print("la precision es:\n", Pre)

la tasa de error es:
 0.21095627102354636
la sensibilidad es:
 0.5528942115768463
la precision es:
 0.5630081300813008


Encontramos que la sensibilidad y la precisión son muy bajas, por lo que podemos decir que al modelo le cuesta identificar correctamente que tipo de autopista fue la del accidente.

In [ ]:
#12 Caso
XS=[[5,1,1,3,4,0]]
mnb.predict(XS)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([0])

El resultado indica que idealmente un accidente de estas características ocurre fuera de carretera